# Strands Agents with AgentCore Memory (Short-Term Memory)


## Introduction

This tutorial demonstrates how to build a **personal agent** using Strands agents with AgentCore **short-term memory** (Raw events). The agent remembers recent conversations in the session using `get_last_k_turns` and can continue conversations seamlessly when user returns.


### Tutorial Details

| Information         | Details                                                                          |
|:--------------------|:---------------------------------------------------------------------------------|
| Tutorial type       | Short Term Conversational                                                        |
| Agent type          | Personal Agent                                                                   |
| Agentic Framework   | Strands Agents                                                                   |
| LLM model           | Anthropic Claude Haiku 4.5                                                      |
| Tutorial components | AgentCore Short-term Memory, AgentInitializedEvent and MessageAddedEvent hooks   |
| Example complexity  | Beginner                                                                         |

You'll learn to:
- Use short-term memory for conversation continuity
- Retrieve last K conversation turns
- Web search tool for real-time information
- Initialize agents with conversation history

## Architecture
<div style="text-align:left">
    <img src="architecture.png" width="65%" />
</div>

## Prerequisites

- Python 3.10+
- AWS credentials with AgentCore Memory permissions
- AgentCore Memory role ARN
- Access to Amazon Bedrock models

Let's get started by setting up our environment!

## Step 1: Setup and Imports

In [1]:
!pip install -qr requirements.txt

In [2]:
import logging
from datetime import datetime

# Setup
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("personal-agent")

In [3]:
# Imports
import os
from strands import Agent, tool
from strands.hooks import AgentInitializedEvent, HookProvider, HookRegistry, MessageAddedEvent
from bedrock_agentcore.memory import MemoryClient

# Configuration
REGION = os.getenv('AWS_REGION', 'us-west-2') # AWS region for the agent
ACTOR_ID = "user_123" # It can be any unique identifier (AgentID, User ID, etc.)
SESSION_ID = "personal_session_001" # Unique session identifier


## Step 2: Web Search Tool

First, let's create a simple web search tool for the agent.

In [4]:
from ddgs.exceptions import DDGSException, RatelimitException
from ddgs import DDGS

@tool
def websearch(keywords: str, region: str = "us-en", max_results: int = 5) -> str:
    """Search the web for updated information.
    
    Args:
        keywords (str): The search query keywords.
        region (str): The search region: wt-wt, us-en, uk-en, ru-ru, etc..
        max_results (int | None): The maximum number of results to return.
    Returns:
        List of dictionaries with search results.
    
    """
    try:
        results = DDGS().text(keywords, region=region, max_results=max_results)
        return results if results else "No results found."
    except RatelimitException:
        return "Rate limit reached. Please try again later."
    except DDGSException as e:
        return f"Search error: {e}"
    except Exception as e:
        return f"Search error: {str(e)}"

logger.info("✅ Web search tool ready")

INFO:personal-agent:✅ Web search tool ready


## Step 3: Create Memory Resource
For short-term memory, we create a memory resource without any strategies. This stores raw conversation turns that can be retrieved with `get_last_k_turns`.


In [5]:
from botocore.exceptions import ClientError

# Initialize Memory Client
client = MemoryClient(region_name=REGION)
memory_name = "PersonalAgentMemory"

try:
    # Create memory resource without strategies (thus only access to short-term memory)
    memory = client.create_memory_and_wait(
        name=memory_name,
        strategies=[],  # No strategies for short-term memory
        description="Short-term memory for personal agent",
        event_expiry_days=7, # Retention period for short-term memory. This can be upto 365 days.
    )
    memory_id = memory['id']
    logger.info(f"✅ Created memory: {memory_id}")
except ClientError as e:
    logger.info(f"❌ ERROR: {e}")
    if e.response['Error']['Code'] == 'ValidationException' and "already exists" in str(e):
        # If memory already exists, retrieve its ID
        memories = client.list_memories()
        memory_id = next((m['id'] for m in memories if m['id'].startswith(memory_name)), None)
        logger.info(f"Memory already exists. Using existing memory ID: {memory_id}")
except Exception as e:
    # Show any errors during memory creation
    logger.error(f"❌ ERROR: {e}")
    import traceback
    traceback.print_exc()
    # Cleanup on error - delete the memory if it was partially created
    if memory_id:
        try:
            client.delete_memory_and_wait(memory_id=memory_id)
            logger.info(f"Cleaned up memory: {memory_id}")
        except Exception as cleanup_error:
            logger.error(f"Failed to clean up memory: {cleanup_error}")

INFO:bedrock_agentcore.memory.client:Initialized MemoryClient for control plane: us-west-2, data plane: us-west-2
INFO:bedrock_agentcore.memory.client:Created memory: PersonalAgentMemory-Wung29GxZx
INFO:bedrock_agentcore.memory.client:Created memory PersonalAgentMemory-Wung29GxZx, waiting for ACTIVE status...
INFO:bedrock_agentcore.memory.client:Memory PersonalAgentMemory-Wung29GxZx is now ACTIVE (took 172 seconds)
INFO:personal-agent:✅ Created memory: PersonalAgentMemory-Wung29GxZx


## Step 4: Memory Hook

This step defines our custom `MemoryHookProvider` class that automates memory operations. Hooks are special functions that run at specific points in an agent's execution lifecycle. The memory hook we're creating serves two primary functions:
1. **To load recent conversation**: We use the `AgentInitializedEvent` hook will automatically load recent conversation history when the agent is initialized.
2. **To store the last message**: Stores new conversational message.

This creates a seamless memory experience without manual management.

In [6]:
class MemoryHookProvider(HookProvider):
    def __init__(self, memory_client: MemoryClient, memory_id: str):
        self.memory_client = memory_client
        self.memory_id = memory_id
    
    def on_agent_initialized(self, event: AgentInitializedEvent):
        """Load recent conversation history when agent starts"""
        try:
            # Get session info from agent state
            actor_id = event.agent.state.get("actor_id")
            session_id = event.agent.state.get("session_id")
            
            if not actor_id or not session_id:
                logger.warning("Missing actor_id or session_id in agent state")
                return
            
            # Load the last 5 conversation turns from memory
            recent_turns = self.memory_client.get_last_k_turns(
                memory_id=self.memory_id,
                actor_id=actor_id,
                session_id=session_id,
                k=5
            )
            
            if recent_turns:
                # Format conversation history for context
                context_messages = []
                for turn in recent_turns:
                    for message in turn:
                        role = message['role']
                        content = message['content']['text']
                        context_messages.append(f"{role}: {content}")
                
                context = "\n".join(context_messages)
                # Add context to agent's system prompt.
                event.agent.system_prompt += f"\n\nRecent conversation:\n{context}"
                logger.info(f"✅ Loaded {len(recent_turns)} conversation turns")
                
        except Exception as e:
            logger.error(f"Memory load error: {e}")
    
    def on_message_added(self, event: MessageAddedEvent):
        """Store messages in memory"""
        messages = event.agent.messages
        try:
            # Get session info from agent state
            actor_id = event.agent.state.get("actor_id")
            session_id = event.agent.state.get("session_id")

            if messages[-1]["content"][0].get("text"):
                self.memory_client.create_event(
                    memory_id=self.memory_id,
                    actor_id=actor_id,
                    session_id=session_id,
                    messages=[(messages[-1]["content"][0]["text"], messages[-1]["role"])]
                )
        except Exception as e:
            logger.error(f"Memory save error: {e}")
    
    def register_hooks(self, registry: HookRegistry):
        # Register memory hooks
        registry.add_callback(MessageAddedEvent, self.on_message_added)
        registry.add_callback(AgentInitializedEvent, self.on_agent_initialized)

## Step 5: Create Personal Agent with Web Search

In [7]:
def create_personal_agent():
    """Create personal agent with memory and web search"""
    agent = Agent(
        name="PersonalAssistant",
        model="global.anthropic.claude-haiku-4-5-20251001-v1:0",  # or your preferred model
        system_prompt=f"""You are a helpful personal assistant with web search capabilities.
        
        You can help with:
        - General questions and information lookup
        - Web searches for current information
        - Personal task management
        
        When you need current information, use the websearch function.
        Today's date: {datetime.today().strftime('%Y-%m-%d')}
        Be friendly and professional.""",
        hooks=[MemoryHookProvider(client, memory_id)],
        tools=[websearch],
        state={"actor_id": ACTOR_ID, "session_id": SESSION_ID}
    )
    return agent

# Create agent
agent = create_personal_agent()
logger.info("✅ Personal agent created with memory and web search")

INFO:bedrock_agentcore.memory.client:Retrieved total of 0 events
INFO:personal-agent:✅ Personal agent created with memory and web search


#### Congratulations ! Your agent is ready ! :) 
## Lets test the Agent

In [8]:
# Test conversation with memory
print("=== First Conversation ===")
print(f"User: My name is Asmsh and I'm interested in learning about AI.")
print(f"Agent: ", end="")
agent("My name is Amsh and I'm interested in learning about AI.")

INFO:bedrock_agentcore.memory.client:Created event: 0000001765650876685#befa07d8
INFO:strands.telemetry.metrics:Creating Strands MetricsClient


=== First Conversation ===
User: My name is Asmsh and I'm interested in learning about AI.
Agent: Hello Amsh! It's great to meet you, and I'm excited to hear that you're interested in learning about AI! 🤖

That's a fascinating field with so much to explore. AI (Artificial Intelligence) is rapidly evolving and touching almost every aspect of our lives. Here are some general areas you might want to explore:

**Key Areas in AI:**
- **Machine Learning** - Teaching computers to learn from data
- **Deep Learning** - Neural networks that mimic how brains work
- **Natural Language Processing (NLP)** - Understanding and generating human language
- **Computer Vision** - Teaching machines to understand images and videos
- **Robotics** - Physical AI systems that interact with the world
- **Large Language Models** - Advanced AI systems like ChatGPT and others

**Getting Started:**
- Online courses (Coursera, edX, Udemy have great AI courses)
- Books like "Artificial Intelligence: A Guide for Thinki

INFO:bedrock_agentcore.memory.client:Created event: 0000001765650880342#4a3414b5


d I'm happy to help! 😊

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': 'Hello Amsh! It\'s great to meet you, and I\'m excited to hear that you\'re interested in learning about AI! 🤖\n\nThat\'s a fascinating field with so much to explore. AI (Artificial Intelligence) is rapidly evolving and touching almost every aspect of our lives. Here are some general areas you might want to explore:\n\n**Key Areas in AI:**\n- **Machine Learning** - Teaching computers to learn from data\n- **Deep Learning** - Neural networks that mimic how brains work\n- **Natural Language Processing (NLP)** - Understanding and generating human language\n- **Computer Vision** - Teaching machines to understand images and videos\n- **Robotics** - Physical AI systems that interact with the world\n- **Large Language Models** - Advanced AI systems like ChatGPT and others\n\n**Getting Started:**\n- Online courses (Coursera, edX, Udemy have great AI courses)\n- Books like "Artificial Intelligence: A Guide fo

In [9]:
print(f"User: Can you search for the latest AI trends in 2025?")
print(f"Agent: ", end="")
agent("Can you search for the latest AI trends in 2025?")

INFO:bedrock_agentcore.memory.client:Created event: 0000001765650908323#caa5e51e


User: Can you search for the latest AI trends in 2025?
Agent: 
Tool #1: websearch


INFO:primp:response: https://en.wikipedia.org/w/api.php?action=opensearch&profile=fuzzy&limit=1&search=latest%20AI%20trends%202025 200
INFO:httpx:HTTP Request: POST https://html.duckduckgo.com/html/ "HTTP/2 200 OK"


Great! I found some excellent information about AI trends in 2025. Here are the key trends shaping the AI landscape right now:

## **Top AI Trends in 2025:**

### **1. Autonomous Agents**
AI systems that can act independently and make decisions without constant human intervention are becoming increasingly sophisticated and practical for real-world applications.

### **2. Open-Source AI**
There's a major shift toward open-source AI models and tools, making AI more accessible and reducing costs for organizations and developers.

### **3. Multi-Model Approaches**
Companies are using multiple AI models together rather than relying on a single model, allowing for more flexible and powerful solutions.

### **4. Cost Optimization**
Focus on creating smaller, more efficient AI models that require less computational power while maintaining performance - making AI more practical and sustainable.

### **5. Orchestration, Governance, and Scale**
Moving beyond pilots and prototypes, 2025 is about a

INFO:bedrock_agentcore.memory.client:Created event: 0000001765650915340#a04592e8


AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': "Great! I found some excellent information about AI trends in 2025. Here are the key trends shaping the AI landscape right now:\n\n## **Top AI Trends in 2025:**\n\n### **1. Autonomous Agents**\nAI systems that can act independently and make decisions without constant human intervention are becoming increasingly sophisticated and practical for real-world applications.\n\n### **2. Open-Source AI**\nThere's a major shift toward open-source AI models and tools, making AI more accessible and reducing costs for organizations and developers.\n\n### **3. Multi-Model Approaches**\nCompanies are using multiple AI models together rather than relying on a single model, allowing for more flexible and powerful solutions.\n\n### **4. Cost Optimization**\nFocus on creating smaller, more efficient AI models that require less computational power while maintaining performance - making AI more practical and sustainable.

In [10]:
print(f"User: I'm particularly interested in machine learning applications.")
print(f"Agent: ", end="")
agent("I'm particularly interested in machine learning applications.")

INFO:bedrock_agentcore.memory.client:Created event: 0000001765650921811#b566809d


User: I'm particularly interested in machine learning applications.
Agent: That's an excellent choice, Amsh! Machine Learning is one of the most practical and impactful areas of AI. Let me search for current information about machine learning applications to give you the most up-to-date insights.
Tool #2: websearch


INFO:bedrock_agentcore.memory.client:Created event: 0000001765650924637#8eee6d0a
INFO:primp:response: https://search.yahoo.com/search;_ylt=0lHZIOD3f2TWdTXDRN5UTPxP;_ylu=n4HAlL3LAu3Xxy224ZmjZBu-eJf0azvjq6qvFz4N8warseE?p=machine+learning+applications+2025+real-world+use+cases 200
INFO:primp:response: https://en.wikipedia.org/w/api.php?action=opensearch&profile=fuzzy&limit=1&search=machine%20learning%20applications%202025%20real-world%20use%20cases 200


Perfect! Here's a comprehensive overview of machine learning applications in 2025:

## **Top Machine Learning Applications in 2025:**

### **Finance & Banking:**
- **Fraud Detection** - ML detects fraudulent transactions in real-time, saving billions in losses
- **Algorithmic Trading** - Uses sentiment analysis and stock price prediction algorithms
- **Credit Risk Assessment** - Predicting loan defaults and creditworthiness
- **Portfolio Management & Robo-Advisors** - Automated financial advisory with personalized recommendations

### **Healthcare:**
- **Disease Diagnosis** - CNN algorithms detect skin cancer with up to 95% accuracy
- **Mortality Risk Prediction** - COVID-19 and other disease outcome predictions
- **Administrative Task Automation** - NLP helps reduce physician burnout by automating paperwork
- **Drug Discovery** - ML accelerates pharmaceutical research

### **Retail & E-Commerce:**
- **Recommendation Engines** - Analyzing millions of customer activities in real-time to

INFO:bedrock_agentcore.memory.client:Created event: 0000001765650932853#97903d1a


 specific application area? 😊

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': "Perfect! Here's a comprehensive overview of machine learning applications in 2025:\n\n## **Top Machine Learning Applications in 2025:**\n\n### **Finance & Banking:**\n- **Fraud Detection** - ML detects fraudulent transactions in real-time, saving billions in losses\n- **Algorithmic Trading** - Uses sentiment analysis and stock price prediction algorithms\n- **Credit Risk Assessment** - Predicting loan defaults and creditworthiness\n- **Portfolio Management & Robo-Advisors** - Automated financial advisory with personalized recommendations\n\n### **Healthcare:**\n- **Disease Diagnosis** - CNN algorithms detect skin cancer with up to 95% accuracy\n- **Mortality Risk Prediction** - COVID-19 and other disease outcome predictions\n- **Administrative Task Automation** - NLP helps reduce physician burnout by automating paperwork\n- **Drug Discovery** - ML accelerates pharmaceutical research\n\n### **Retail 

## Test Memory Continuity

To test if our memory system is working correctly, we'll create a new instance of the agent and see if it can access the previously stored information:

In [11]:
# Create new agent instance (simulates user returning)
print("=== User Returns - New Session ===")
new_agent = create_personal_agent()

# Test memory continuity
print(f"User: What was my name again?")
print(f"Agent: ", end="")
new_agent("What was my name again?")

print(f"User: Can you search for more information about machine learning?")
print(f"Agent: ", end="")
new_agent("Can you search for more information about machine learning?")

INFO:bedrock_agentcore.memory.client:Retrieved total of 7 events
INFO:personal-agent:✅ Loaded 4 conversation turns


=== User Returns - New Session ===
User: What was my name again?
Agent: 

INFO:bedrock_agentcore.memory.client:Created event: 0000001765650946368#1d8d35f0


Your name is **Amsh**! 😊

You mentioned it at the beginning of our conversation when you said you're interested in learning about AI. Is there anything else I can help you with regarding AI or machine learning?

INFO:bedrock_agentcore.memory.client:Created event: 0000001765650948670#a4d4d827
INFO:bedrock_agentcore.memory.client:Created event: 0000001765650948813#68f3d84b


User: Can you search for more information about machine learning?
Agent: 
Tool #1: websearch


INFO:primp:response: https://en.wikipedia.org/w/api.php?action=opensearch&profile=fuzzy&limit=1&search=machine%20learning%20applications%202025 200
INFO:primp:response: https://www.mojeek.com/search?q=machine+learning+applications+2025 403
INFO:primp:response: https://yandex.com/search/site/?text=machine+learning+applications+2025&web=1&searchid=8667986 200


Great! Here's the latest information about machine learning in 2025:

## **Machine Learning Market Overview:**

### **Market Size & Growth:**
- The Machine Learning AI market was valued at **$7.59 Billion in 2025**
- Showing steady adoption across multiple industries and geographies
- Significant expansion expected as businesses and governments continue to invest

## **Latest Machine Learning Trends in 2025:**

### **1. Vibe Coding**
A cutting-edge AI-assisted software development technique that's gaining popularity. It was even named **Collins Dictionary's Word of the Year for 2025**! This represents how AI is now integrated into the coding and development process itself.

### **2. Quantum Machine Learning**
- CSIRO (Australian research organization) is demonstrating practical applications for quantum machine learning
- Includes technologies like Self-Adaptive Quantum Kernel (SAQK) for advanced data processing
- This represents the next frontier in computational power for ML

### **3.

INFO:bedrock_agentcore.memory.client:Created event: 0000001765650958250#8ff6da54


AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': "Great! Here's the latest information about machine learning in 2025:\n\n## **Machine Learning Market Overview:**\n\n### **Market Size & Growth:**\n- The Machine Learning AI market was valued at **$7.59 Billion in 2025**\n- Showing steady adoption across multiple industries and geographies\n- Significant expansion expected as businesses and governments continue to invest\n\n## **Latest Machine Learning Trends in 2025:**\n\n### **1. Vibe Coding**\nA cutting-edge AI-assisted software development technique that's gaining popularity. It was even named **Collins Dictionary's Word of the Year for 2025**! This represents how AI is now integrated into the coding and development process itself.\n\n### **2. Quantum Machine Learning**\n- CSIRO (Australian research organization) is demonstrating practical applications for quantum machine learning\n- Includes technologies like Self-Adaptive Quantum Kernel (SAQK) 

## View Stored Memory

In [14]:
# Check what's stored in memory
print("=== Memory Contents ===")
recent_turns = client.get_last_k_turns(
    memory_id=memory_id,
    actor_id=ACTOR_ID,
    session_id=SESSION_ID,
    k=10 # Adjust k to see more or fewer turns
)

for i, turn in enumerate(recent_turns, 1):
    print(f"Turn {i}:")
    for message in turn:
        role = message['role']
        content = message['content']['text'][:100] + "..." if len(message['content']['text']) > 100 else message['content']['text']
        print(f"  {role}: {content}")
    print()

INFO:bedrock_agentcore.memory.client:Retrieved total of 11 events


=== Memory Contents ===
Turn 1:
  ASSISTANT: Great! Here's the latest information about machine learning in 2025:

## **Machine Learning Market O...

Turn 2:
  USER: Can you search for more information about machine learning?
  ASSISTANT: Your name is **Amsh**! 😊

You mentioned it at the beginning of our conversation when you said you're...

Turn 3:
  USER: What was my name again?
  ASSISTANT: Perfect! Here's a comprehensive overview of machine learning applications in 2025:

## **Top Machine...
  ASSISTANT: That's an excellent choice, Amsh! Machine Learning is one of the most practical and impactful areas ...

Turn 4:
  USER: I'm particularly interested in machine learning applications.
  ASSISTANT: Great! I found some excellent information about AI trends in 2025. Here are the key trends shaping t...

Turn 5:
  USER: Can you search for the latest AI trends in 2025?
  ASSISTANT: Hello Amsh! It's great to meet you, and I'm excited to hear that you're interested in learning about...

Tu

## Summary

This tutorial showed how to build a personal agent. You've learned:

- Creating memory resources without strategies
- Using `get_last_k_turns` for conversation history
- Adding web search capabilities to agents
- Implementing memory hooks for context loading

**Next Steps:**
- Add more sophisticated tools
- Implement long-term memory strategies
- Enhance search capabilities with multiple sources

## Cleanup (Optional)

In [ ]:
# Uncomment to delete memory resource
# client.delete_memory_and_wait(memory_id)
# logger.info(f"✅ Deleted memory: {memory_id}")